### Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Engine

#### 1. Import data from URL into Pandas DataFrame

In [2]:
data_url = 'https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks'

page = BeautifulSoup(requests.get(url=data_url).text, 'lxml')
table = page.find('table', attrs={'class': 'wikitable'})
df = pd.read_html(str(table))[0]
df

,Rank,Bank name,Market cap (US$ billion)
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91
5,6,Wells Fargo,155.87
6,7,HSBC Holdings PLC,148.90
7,8,Morgan Stanley,140.83
8,9,China Construction Bank,139.82
9,10,Bank of China,136.81


#### 2. Import data from .CSV into Pandas DataFrame

In [3]:
rates_file_path = '../files/exchange_rate.csv'
df_rates = pd.read_csv(rates_file_path)
df_rates

,Currency,Rate
0,EUR,0.93
1,GBP,0.80
2,INR,82.95


#### 3. Treats the data as needed

In [4]:
df = df.rename(columns={'Market cap (US$ billion)': 'MC_USD_Billion'})
eur_value = float(df_rates.loc[df_rates['Currency'] == 'EUR', 'Rate'].iloc[0])
gbp_value = float(df_rates.loc[df_rates['Currency'] == 'GBP', 'Rate'].iloc[0])
inr_value = float(df_rates.loc[df_rates['Currency'] == 'INR', 'Rate'].iloc[0])
df['MC_EUR_Billion'] = np.round(df['MC_USD_Billion'] * eur_value, 2)
df['MC_GBP_Billion'] = np.round(df['MC_USD_Billion'] * gbp_value, 2)
df['MC_INR_Billion'] = np.round(df['MC_USD_Billion'] * inr_value, 2)

df

,Rank,Bank name,MC_USD_Billion,MC_EUR_Billion,MC_GBP_Billion,MC_INR_Billion
0,1,JPMorgan Chase,432.92,402.62,346.34,35910.71
1,2,Bank of America,231.52,215.31,185.22,19204.58
2,3,Industrial and Commercial Bank of China,194.56,180.94,155.65,16138.75
3,4,Agricultural Bank of China,160.68,149.43,128.54,13328.41
4,5,HDFC Bank,157.91,146.86,126.33,13098.63
5,6,Wells Fargo,155.87,144.96,124.70,12929.42
6,7,HSBC Holdings PLC,148.90,138.48,119.12,12351.26
7,8,Morgan Stanley,140.83,130.97,112.66,11681.85
8,9,China Construction Bank,139.82,130.03,111.86,11598.07
9,10,Bank of China,136.81,127.23,109.45,11348.39


#### 4. Exports the data into .csv and .db

In [5]:
save_csv_path = '../files/Largest_banks_data.csv'
table_name = 'Largest_banks'
engine = create_engine('sqlite:///../files/Banks.db')

df.to_csv(save_csv_path)
df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

10

#### 5. Executes several example queries

In [6]:
def run_query(engine: Engine, statment: str) -> pd.DataFrame:
    query_output = pd.read_sql(statment, engine)
    print(query_output)
    return query_output

In [7]:
df1 = run_query(engine=engine, statment='SELECT * FROM Largest_banks')
df1

   Rank                                Bank name  MC_USD_Billion  \
0     1                           JPMorgan Chase          432.92   
1     2                          Bank of America          231.52   
2     3  Industrial and Commercial Bank of China          194.56   
3     4               Agricultural Bank of China          160.68   
4     5                                HDFC Bank          157.91   
5     6                              Wells Fargo          155.87   
6     7                        HSBC Holdings PLC          148.90   
7     8                           Morgan Stanley          140.83   
8     9                  China Construction Bank          139.82   
9    10                            Bank of China          136.81   

   MC_EUR_Billion  MC_GBP_Billion  MC_INR_Billion  
0          402.62          346.34        35910.71  
1          215.31          185.22        19204.58  
2          180.94          155.65        16138.75  
3          149.43          128.54        13

,Rank,Bank name,MC_USD_Billion,MC_EUR_Billion,MC_GBP_Billion,MC_INR_Billion
0,1,JPMorgan Chase,432.92,402.62,346.34,35910.71
1,2,Bank of America,231.52,215.31,185.22,19204.58
2,3,Industrial and Commercial Bank of China,194.56,180.94,155.65,16138.75
3,4,Agricultural Bank of China,160.68,149.43,128.54,13328.41
4,5,HDFC Bank,157.91,146.86,126.33,13098.63
5,6,Wells Fargo,155.87,144.96,124.70,12929.42
6,7,HSBC Holdings PLC,148.90,138.48,119.12,12351.26
7,8,Morgan Stanley,140.83,130.97,112.66,11681.85
8,9,China Construction Bank,139.82,130.03,111.86,11598.07
9,10,Bank of China,136.81,127.23,109.45,11348.39


In [8]:
df2 = run_query(engine=engine, statment='SELECT AVG(MC_GBP_Billion) FROM Largest_banks')
df2

   AVG(MC_GBP_Billion)
0              151.987


,AVG(MC_GBP_Billion)
0,151.987


In [9]:
df3 = run_query(engine=engine, statment='SELECT `Bank name` from Largest_banks LIMIT 5')
df3

                                 Bank name
0                           JPMorgan Chase
1                          Bank of America
2  Industrial and Commercial Bank of China
3               Agricultural Bank of China
4                                HDFC Bank


,Bank name
0,JPMorgan Chase
1,Bank of America
2,Industrial and Commercial Bank of China
3,Agricultural Bank of China
4,HDFC Bank
